# Classical Search

This notebook serves as a supporting material for the chapter **Solving Problems by Searching**. The notebooks illustrate the use of the code repository and demonstrate how the code can be extended to solve various search related problems. The discussion of problem solving begins with a precise implementation of **problems** and their **solutions**. Then we move onto various **informed** and **uninformed** search strategies for solving problems.

In [16]:
%classpath add jar ../out/artifacts/aima_core_jar/aima-core.jar

## Problem Solving Agents

The process of looking for a sequence of **actions** that reaches the **goal** is called **search**.
A search algorithm takes a problem as input and returns a solution in the form of an action
sequence. Once a solution is found, the actions it recommends can be carried out. This
is called the execution phase. Thus, we have a simple “formulate, search, execute” design
for the agent, as shown in Figure 3.1 of the textbook. After formulating a goal and a problem to solve,
the agent calls a search procedure to solve it. It then uses the solution to guide its actions,
doing whatever the solution recommends as the next thing to do—typically, the first action of
the sequence—and then removing that step from the sequence. Once the solution has been
executed, the agent will formulate a new goal.

Let's have a look at the pseudocode of a simple problem solving agent and then see it's java implementation.

In [7]:
%%python
from notebookUtils import *
pseudocode('Simple Problem Solving Agent')

### AIMA3e
__function__ SIMPLE-PROBLEM-SOLVING-AGENT(_percept_) __returns__ an action  
&emsp;__persistent__: _seq_, an action sequence, initially empty  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;_state_, some description of the current world state  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;_goal_, a goal, initially null  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;_problem_, a problem formulation

&emsp;_state_ &larr; UPDATE-STATE(_state_, _percept_)  
&emsp;__if__ _seq_ is empty __then__  
&emsp;&emsp;&emsp;_goal_ &larr; FORMULATE-GOAL(_state_)  
&emsp;&emsp;&emsp;_problem_ &larr; FORMULATE-PROBLEM(_state_, _goal_)  
&emsp;&emsp;&emsp;_seq_ &larr; SEARCH(_problem_)  
&emsp;&emsp;&emsp;__if__ _seq_ = _failure_ __then return__ a null action  
&emsp;_action_ &larr; FIRST(_seq_)  
&emsp;_seq_ &larr; REST(_seq_)  
&emsp;__return__ _action_  

---
__Figure__ ?? A simple problem-solving agent. It first formulates a goal and a problem, searches for a sequence of actions that would solve the problem, and then executes the actions one at a time. When this is complete, it formulates another goal and starts over.

The implementation of the above pseudocode can be viewed [here](https://github.com/aimacode/aima-java/blob/AIMA3e/aima-core/src/main/java/aima/core/search/agent/SimpleProblemSolvingAgent.java). This agent is implemented as an abstract agent which can be extended to construct other agents.

### Well-defined problems and solutions
We will first formally define a problem. Then, we will have a look at how the code from the repository can be used to formulate new problems. Then, we will have a look at various toy problems which are already present in the code repository.

As per the textbook, a **problem** can be defined formally by five components. The initial state, applicable actions, the transition model, the goal test and the path cost function. This five component structure is implemented as an interface named [Problem.java]() in the repository. Let's have a look at the implementation

````java
public interface Problem<S, A> extends OnlineSearchProblem<S, A> {

    /**
     * Returns the initial state of the agent.
     */
    S getInitialState();

    /**
     * Returns the description of the possible actions available to the agent.
     */
    List<A> getActions(S state);

    /**
     * Returns the description of what each action does.
     */
    S getResult(S state, A action);

    /**
     * Determines whether a given state is a goal state.
     */
    boolean testGoal(S state);

    /**
     * Returns the <b>step cost</b> of taking action <code>action</code> in state <code>state</code> to reach state
     * <code>stateDelta</code> denoted by c(s, a, s').
     */
    double getStepCosts(S state, A action, S stateDelta);

    /**
     * Tests whether a node represents an acceptable solution. The default implementation
     * delegates the check to the goal test. Other implementations could make use of the additional
     * information given by the node (e.g. the sequence of actions leading to the node). A
     * solution tester implementation could for example always return false and internally collect
     * the paths of all nodes whose state passes the goal test. Search implementations should always
     * access the goal test via this method to support solution acceptance testing.
     */
    default boolean testSolution(Node<S, A> node) {
        return testGoal(node.getState());
    }
}
````

The states and actions are represented by the generic variables `S` and `A` respectively. Clearly, the methods represent the
five components of a particular problem as follows:
* initial state &larr; `getInitialState()`
* applicable actions &larr; `getActions(S state)`
* the transition model &larr; `getResult(S state, A action)`
* the goal test &larr; `testGoal(S state)`
* path cost function &larr; `getStepCosts(S state, A action, S stateDelta)`

## Example Problems

A **toy problem** is intended to illustrate or exercise various problem solving methods. Let's extend the `Problem` interface to implement a toy problem. Let's implement the **8 puzzle problem** which consists of a 3x3 board with eight numbered tiles and a blank space. It has the following five components:
* **States**: A state description specifies the location of each of the eight tiles and the blank in one of the nine squares.
* **Initial state**: Any state can be designated as the initial state. Note that any given goal can be reached from exactly half of the possible initial states. (proved in Exercise 3.4)
* **Actions**: The simplest formulation defines the actions as movements of the blank space Left, Right, Up, or Down. Different subsets of these are possible depending on where the blank is.
* **Transition model**: Given a state and action, this returns the resulting state.
* **Path cost**: Each step costs 1, so the path cost is the number of steps in the path.

Let's look at the implementation:


First we implement the states and actions applicable to the problem. The actions can be implemented as an enum whereas the states can be represented as an array of ints.

In [50]:
package aima.notebooks.classicalsearch;

import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;
import aima.core.search.framework.problem.Problem;

public class EightPuzzleProblem implements Problem<int[], EightPuzzleProblem.Action> {

    int[] initialState = new int[9];
    
    public EightPuzzleProblem(int[] initialState){
        this.initialState = initialState;
    }

    @Override
    public int[] getInitialState() {
        return initialState;
    }

    @Override
    public List<Action> getActions(int[] state) {
        List<Action> actions = new ArrayList<>();
        if (this.canMoveGap(state, Action.UP))
            actions.add(Action.UP);
        if (this.canMoveGap(state, Action.DOWN))
            actions.add(Action.DOWN);
        if (this.canMoveGap(state, Action.LEFT))
            actions.add(Action.LEFT);
        if (this.canMoveGap(state, Action.RIGHT))
            actions.add(Action.RIGHT);
        return actions;
    }

    @Override
    public int[] getResult(int[] state, Action action) {
        int[] result = state.clone();

        if (Action.UP.equals(action) && canMoveGap(state, Action.UP))
            moveGapUp(result);
        else if (Action.DOWN.equals(action) && canMoveGap(state, Action.DOWN))
            moveGapDown(result);
        else if (Action.LEFT.equals(action) && canMoveGap(state, Action.LEFT))
            moveGapLeft(result);
        else if (Action.RIGHT.equals(action) && canMoveGap(state, Action.RIGHT))
            moveGapRight(result);
        return result;
    }

    private void moveGapRight(int[] result) {
        int gapPos = getGapPosition(result);
        int x = gapPos / 3;
        int y = gapPos % 3;
        if (!(y == 2)) {
            int valueOnRight = result[x * 3 + y + 1];
            setValue(result, x, y, valueOnRight);
            setValue(result, x, y + 1, 0);
        }
    }

    private void moveGapLeft(int[] result) {
        int gapPos = getGapPosition(result);
        int x = gapPos / 3;
        int y = gapPos % 3;
        if (!(y == 0)) {
            int valueOnLeft = result[x * 3 + (y - 1)];
            setValue(result, x, y, valueOnLeft);
            setValue(result, x, y - 1, 0);
        }
    }

    private void moveGapDown(int[] result) {
        int gapPos = getGapPosition(result);
        int x = gapPos / 3;
        int y = gapPos % 3;
        if (!(x == 2)) {
            int valueOnBottom = result[(x + 1) * 3 + y];
            System.out.println(valueOnBottom);
            setValue(result, x, y, valueOnBottom);
            setValue(result, x + 1, y, 0);
        }
    }

    @Override
    public boolean testGoal(int[] state) {
        return Arrays.equals(state, new int[]{0, 1, 2, 3, 4, 5, 6, 7, 8});
    }

    @Override
    public double getStepCosts(int[] state, Action action, int[] stateDelta) {
        return 1.0;
    }

    private int getGapPosition(int[] state) {
        return getPositionOf(state, 0);
    }

    private int getPositionOf(int[] state, int val) {
        for (int i = 0; i < 9; i++)
            if (state[i] == val)
                return i;
        return -1;
    }

    public boolean canMoveGap(int[] state, Action where) {
        boolean retVal = true;
        int absPos = getPositionOf(state, 0);
        if (where.equals(Action.LEFT))
            retVal = (absPos % 3 != 0);
        else if (where.equals(Action.RIGHT))
            retVal = (absPos % 3 != 2);
        else if (where.equals(Action.UP))
            retVal = ((absPos / 3) != 0);
        else if (where.equals(Action.DOWN))
            retVal = ((absPos / 3) != 2);
        return retVal;
    }

    public void moveGapUp(int[] result) {
        int gapPos = getGapPosition(result);
        int x = gapPos / 3;
        int y = gapPos % 3;
        if (!(x == 0)) {
            int valueOnTop = result[(x - 1) * 3 + y];
            setValue(result, x, y, valueOnTop);
            setValue(result, x - 1, y, 0);
        }
    }

    private void setValue(int[] result, int x, int y, int valueOnTop) {
        int absPos = x *3 + y;
        result[absPos] = valueOnTop;
    }

    public enum Action {
        LEFT, RIGHT, UP, DOWN
    }
}



aima.notebooks.classicalsearch.EightPuzzleProblem

So, in this way we can implement a Problem. Now let us see our problem class in action

In [51]:
import aima.notebooks.classicalsearch.EightPuzzleProblem;
import java.util.*;

int [] initialState = new int[] { 5, 4, 0, 6, 1, 8, 7, 3, 2 };

EightPuzzleProblem problem = new EightPuzzleProblem(initialState);

System.out.println("Initial State = " + Arrays.toString(initialState));
System.out.println("Available Actions = " + problem.getActions(initialState).toString());
System.out.println("Resulting State = " + Arrays.toString(problem.getResult(initialState,problem.getActions(initialState).get(0))));
System.out.println("isGoal\t"+ problem.testGoal(initialState));

Initial State = [5, 4, 0, 6, 1, 8, 7, 3, 2]
Available Actions = [DOWN, LEFT]
8
Resulting State = [5, 4, 8, 6, 1, 0, 7, 3, 2]
isGoal	false


null